# Before starting:
* This notebook is meant to run on Google Colaboratory;
* It requires you to download a third party software, otherwise it will not run. (I know it's tedious but I don't own the program and so I can't share it with you);
* I suggest to use an environment with a GPU.


## Preliminaries

### Install 'tensorflow-model-optimization' for Quantization Aware Training (QAT)

### Get 'stm32tflm' software

* Download 'X-CUBE-AI-Linux' package from https://www.st.com/en/embedded-software/x-cube-ai.html;
* Extract the 'stm32tflm' executable from the downloaded package;
* Put it in the folder you're working (usually '/content/' for Google Colaboratory).

## How to use GPU in local Tensorflow fow windows

https://www.tensorflow.org/install/pip#windows-native_1

conda create --name tf python=3.9

conda deactivate
conda activate tf

conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0

pip install --upgrade pip

pip install "tensorflow<2.11" 

python -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

Enable the execution of the 'stm32tflm' program

In [11]:
from tensorflow import keras
from pathlib import Path
import tensorflow as tf
import numpy as np
import subprocess
import datetime
import shutil
import glob
import re
import os

In [12]:
# Check for available GPU devices
gpu_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu_devices))

if gpu_devices:
    print("Found GPU(s):", gpu_devices)
else:
    print("No GPU devices found. TensorFlow is likely using the CPU.")

Num GPUs Available:  1
Found GPU(s): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## ColabNAS code

In [65]:
class ColabNAS :
    architecture_name = 'resulting_architecture'
    def __init__(self, max_RAM, max_Flash, max_MACC, path_to_training_set, val_split, cache=False, input_shape=(50,50,3), save_path='.', path_to_stm32tflm='stm32tflm.exe') :
        self.learning_rate = 1e-3
        self.batch_size = 128
        self.epochs = 100 #minimum 2

        self.max_MACC = max_MACC
        self.max_Flash = max_Flash
        self.max_RAM = max_RAM
        self.path_to_training_set = path_to_training_set
        self.num_classes = len(next(os.walk(path_to_training_set))[1])
        self.val_split = val_split
        self.cache = cache
        self.input_shape = input_shape
        self.save_path = Path(save_path)

        self.path_to_trained_models = self.save_path / "trained_models"
        self.path_to_trained_models.mkdir(parents=True, exist_ok=True)

        self.path_to_stm32tflm = Path(path_to_stm32tflm)

        self.load_training_set()

    def load_training_set(self):
        if 3 == self.input_shape[2] :
            color_mode = 'rgb'
        elif 1 == self.input_shape[2] :
            color_mode = 'grayscale'

        train_ds = tf.keras.utils.image_dataset_from_directory(
            directory= self.path_to_training_set,
            labels='inferred',
            label_mode='categorical',
            color_mode=color_mode,
            batch_size=self.batch_size,
            image_size=self.input_shape[0:2],
            shuffle=True,
            seed=11,
            validation_split=self.val_split,
            subset='training'
        )

        validation_ds = tf.keras.utils.image_dataset_from_directory(
            directory= self.path_to_training_set,
            labels='inferred',
            label_mode='categorical',
            color_mode=color_mode,
            batch_size=self.batch_size,
            image_size=self.input_shape[0:2],
            shuffle=True,
            seed=11,
            validation_split=self.val_split,
            subset='validation'
        )

        data_augmentation = tf.keras.Sequential([
            tf.keras.layers.RandomFlip("horizontal"),
            tf.keras.layers.RandomRotation(0.2, fill_mode='constant', interpolation='bilinear'),
            #tf.keras.layers.Rescaling(1./255)
            ])

        if self.cache :
            self.train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
            self.validation_ds = validation_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
        else :
            self.train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE).prefetch(buffer_size=tf.data.AUTOTUNE)
            self.validation_ds = validation_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

    def get_data(self):
        return self.train_ds, self.validation_ds

    def quantize_model_uint8(self, model_name) :
        def representative_dataset():
            count = 0
            for images, labels in self.train_ds:
                for i in range(images.shape[0]):
                    if count >= 150:
                        return
                    # Expand dims to make it (1, H, W, C)
                    yield [tf.dtypes.cast(images[i:i+1], tf.float32)]
                    count += 1

        model = tf.keras.models.load_model(self.path_to_trained_models / f"{model_name}.h5")
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_dataset
        converter.target_spec.supported_types = [tf.int8]
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.uint8
        converter.inference_output_type = tf.uint8
        tflite_quant_model = converter.convert()

        with open(self.path_to_trained_models / f"{model_name}.tflite", 'wb') as f:
            f.write(tflite_quant_model)

        (self.path_to_trained_models / f"{model_name}.h5").unlink()

    def evaluate_flash_and_peak_RAM_occupancy(self, model_name) :
        #quantize model to evaluate its peak RAM occupancy and its Flash occupancy
        self.quantize_model_uint8(model_name)

        #evaluate its peak RAM occupancy and its Flash occupancy using STMicroelectronics' X-CUBE-AI
        proc = subprocess.Popen([self.path_to_stm32tflm, self.path_to_trained_models / f"{model_name}.tflite"], stdout=subprocess.PIPE)
        try:
            outs, errs = proc.communicate(timeout=15)
            Flash, RAM = re.findall(r'\d+', str(outs))
        except subprocess.TimeoutExpired:
            proc.kill()
            outs, errs = proc.communicate()
            print("stm32tflm error")
            exit()

        return int(Flash), int(RAM)

    def evaluate_model(self, model, MACC, number_of_cells_limited, model_name) :
        print(f"\n{model_name}\n")
        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            str(self.path_to_trained_models / f"{model_name}.h5"), monitor='val_accuracy',
            verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
        #One epoch of training must be done before quantization, which is needed to evaluate RAM and Flash occupancy
        model.fit(self.train_ds, epochs=1, validation_data=self.validation_ds, validation_freq=1)
        model.save(self.path_to_trained_models / f"{model_name}.h5")
        Flash, RAM = self.evaluate_flash_and_peak_RAM_occupancy(model_name)
        print(f"\nRAM: {RAM},\t Flash: {Flash},\t MACC: {MACC}\n")
        if MACC <= self.max_MACC and Flash <= self.max_Flash and RAM <= self.max_RAM and not number_of_cells_limited :
            hist = model.fit(self.train_ds, epochs=self.epochs - 1, validation_data=self.validation_ds, validation_freq=1, callbacks=[checkpoint])
            self.quantize_model_uint8(model_name)
            return {'RAM': RAM,
                    'Flash': Flash,
                    'MACC': MACC,
                    'max_val_acc':
                    np.around(np.amax(hist.history['val_accuracy']), decimals=3)
                    if 'hist' in locals() else -3}
        else :
            return {'max_val_acc':0}

    def search(self, NAS):
      nas = NAS(
        evaluate_model_fnc = self.evaluate_model, 
        input_shape = self.input_shape, 
        num_classes = self.num_classes, 
        learning_rate = self.learning_rate
        )
      resulting_architecture, take_time = nas.search()

      if (resulting_architecture['max_val_acc'] > 0) :
            resulting_architecture_name = f"k_{resulting_architecture['k']}_c_{resulting_architecture['c']}.tflite"
            self.path_to_resulting_architecture = self.save_path / f"resulting_architecture_{resulting_architecture_name}"
            (self.path_to_trained_models / f"{resulting_architecture_name}").rename(self.path_to_resulting_architecture)
            shutil.rmtree(self.path_to_trained_models)
            print(f"\nResulting architecture: {resulting_architecture}\n")
      else :
          print(f"\nNo feasible architecture found\n")
      print(f"Elapsed time (search): {take_time}\n")

      return self.path_to_resulting_architecture

## First NAS

In [ ]:
class OurNAS():
  architecture_name = 'resulting_architecture'
  def __init__(self, evaluate_model_fnc, input_shape, num_classes, learning_rate):
    self.evaluate_model_fnc = evaluate_model_fnc
    self.model_count = 0
    self.model_name = ""
    self.input_shape = input_shape
    self.num_classes = num_classes
    self.learning_rate = learning_rate

  def create_model(self, k, c):
    kernel_size = (3,3)
    pool_size = (2,2)
    pool_strides = (2,2)

    number_of_cells_limited = False
    number_of_mac = 0

    inputs = keras.Input(shape=self.input_shape)

    #convolutional base
    n = int(k)
    multiplier = 2

    #first convolutional layer
    c_in = self.input_shape[2]
    x = keras.layers.Conv2D(n, kernel_size, padding='same')(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)

    number_of_mac = number_of_mac + (c_in * kernel_size[0] * kernel_size[1] * x.shape[1] * x.shape[2] * x.shape[3])

    #adding cells
    for i in range(1, c + 1) :
        if x.shape[1] <= 1 or x.shape[2] <= 1 :
            number_of_cells_limited = True
            break;
        n = int(np.ceil(n * multiplier))
        multiplier = multiplier - 2**-i
        x = keras.layers.MaxPooling2D(pool_size=pool_size, strides=pool_strides, padding='valid')(x)
        c_in = x.shape[3]
        x = keras.layers.Conv2D(n, kernel_size, padding='same')(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.ReLU()(x)
        number_of_mac = number_of_mac + (c_in * kernel_size[0] * kernel_size[1] * x.shape[1] * x.shape[2] * x.shape[3])

    #classifier
    x = keras.layers.GlobalAveragePooling2D()(x)
    input_shape = x.shape[1]
    x = keras.layers.Dense(n)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    number_of_mac = number_of_mac + (input_shape * x.shape[1])
    x = keras.layers.Dense(self.num_classes)(x)
    x = keras.layers.BatchNormalization()(x)
    outputs = keras.layers.Softmax()(x)
    number_of_mac = number_of_mac + (x.shape[1] * outputs.shape[1])

    model = keras.Model(inputs=inputs, outputs=outputs)

    opt = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
    model.compile(optimizer=opt,
            loss='categorical_crossentropy',
            metrics=['accuracy'])

    model.summary()

    return model, number_of_mac, number_of_cells_limited

  def search(self):
    self.model_counter = 0
    epsilon = 0.005
    k0 = 4

    start = datetime.datetime.now()

    k = k0
    previous_architecture = self.explore_num_cells(k)
    k = 2 * k
    current_architecture = self.explore_num_cells(k)

    if (current_architecture['max_val_acc'] > previous_architecture['max_val_acc']) :
        previous_architecture = current_architecture
        k = 2 * k
        current_architecture = self.explore_num_cells(k)
        while(current_architecture['max_val_acc'] > previous_architecture['max_val_acc'] + epsilon) :
            previous_architecture = current_architecture
            k = 2 * k
            current_architecture = self.explore_num_cells(k)
    else :
        k = k0 / 2
        current_architecture = self.explore_num_cells(k)
        while(current_architecture['max_val_acc'] >= previous_architecture['max_val_acc']) :
            previous_architecture = current_architecture
            k = k / 2
            current_architecture = self.explore_num_cells(k)

    resulting_architecture = previous_architecture
    end = datetime.datetime.now()

    return resulting_architecture, end-start

  def explore_num_cells(self, k) :
      previous_architecture = {'k': -1, 'c': -1, 'max_val_acc': -2}
      current_architecture = {'k': -1, 'c': -1, 'max_val_acc': -1}
      c = -1
      k = int(k)

      while(current_architecture['max_val_acc'] > previous_architecture['max_val_acc']) :
          previous_architecture = current_architecture
          c = c + 1
          self.model_counter = self.model_counter + 1
          current_architecture = self.evaluate_model_process(k, c)
          print(f"\n\n\n{current_architecture}\n\n\n")
      return previous_architecture


  def evaluate_model_process(self, k, c):
    if k > 0 :
      self.model_name = f"k_{k}_c_{c}"
      model, MACC, number_of_cells_limited = self.create_model(k, c)
      result_property_dict = self.evaluate_model_fnc(model, MACC, number_of_cells_limited, self.model_name)
      result_property_dict["k"] = k
      result_property_dict["c"] = c if not number_of_cells_limited else "Not feasible"
      return result_property_dict
    else :
      return{'k': 'unfeasible', 'c': c, 'max_val_acc': -3}

### download a small dataset

In [6]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos.tar', origin=dataset_url, extract=True)
data_dir = Path(data_dir).with_suffix('')

### Run HW-NAS
target: STM32L412KBU3 (273 CoreMark, 40 kiB RAM, 128 kiB Flash)

In [ ]:
input_shape = (50,50,3)

#target: STM32L412KBU3
#273 CoreMark, 40 kiB RAM, 128 kiB Flash
peak_RAM_upper_bound = 40960
Flash_upper_bound = 131072
MACC_upper_bound = 2730000 #CoreMark * 1e4

#Each dataset must comply with the following structure
#main_directory/
#...class_a/
#......a_image_1.jpg
#......a_image_2.jpg
#...class_b/
#......b_image_1.jpg
#......b_image_2.jpg
path_to_training_set = data_dir
val_split = 0.3

#whether or not to cache datasets in memory
#if the dataset cannot fit in the main memory, the application will crash
cache = True

#where to save results
save_path = ''

#to show the GPU used
!nvidia-smi

colabNAS = ColabNAS(peak_RAM_upper_bound, Flash_upper_bound, MACC_upper_bound, path_to_training_set, val_split, cache, input_shape, save_path=save_path)

#search
path_to_tflite_model = colabNAS.search(OurNAS)

Tue Jan  6 00:11:11 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 591.59                 Driver Version: 591.59         CUDA Version: 13.1     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
| 47%   53C    P8             34W /  350W |    1194MiB /  12288MiB |     31%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

INFO:tensorflow:Assets written to: C:\Users\Jimmy\AppData\Local\Temp\tmpd7_y43n0\assets


INFO:tensorflow:Assets written to: C:\Users\Jimmy\AppData\Local\Temp\tmpd7_y43n0\assets
c:\Users\Jimmy\miniconda3\envs\tf\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "



RAM: 19968,	 Flash: 3904,	 MACC: 270041

Epoch 1/99
20/21 [===========================>..] - ETA: 0s - loss: 1.6461 - accuracy: 0.2418
Epoch 1: val_accuracy improved from -inf to 0.18256, saving model to trained_models\k_4_c_0.h5
21/21 [==============================] - 0s 17ms/step - loss: 1.6451 - accuracy: 0.2421 - val_loss: 1.5867 - val_accuracy: 0.1826
Epoch 2/99
15/21 [====================>.........] - ETA: 0s - loss: 1.6142 - accuracy: 0.2792
Epoch 2: val_accuracy improved from 0.18256 to 0.19074, saving model to trained_models\k_4_c_0.h5
21/21 [==============================] - 0s 10ms/step - loss: 1.5926 - accuracy: 0.2880 - val_loss: 1.5870 - val_accuracy: 0.1907
Epoch 3/99
15/21 [====================>.........] - ETA: 0s - loss: 1.5712 - accuracy: 0.3083
Epoch 3: val_accuracy improved from 0.19074 to 0.21435, saving model to trained_models\k_4_c_0.h5
21/21 [==============================] - 0s 10ms/step - loss: 1.5494 - accuracy: 0.3157 - val_loss: 1.5697 - val_accuracy: 0.

INFO:tensorflow:Assets written to: C:\Users\Jimmy\AppData\Local\Temp\tmps_s6nnqq\assets


...




{'RAM': 31232, 'Flash': 12656, 'MACC': 1758265, 'max_val_acc': 0.608, 'k': 8, 'c': 2}



Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 50, 50, 3)]       0         
                                                                 
 conv2d_16 (Conv2D)          (None, 50, 50, 16)        448       
                                                                 
 batch_normalization_30 (Bat  (None, 50, 50, 16)       64        
 chNormalization)                                                
                                                                 
 re_lu_23 (ReLU)             (None, 50, 50, 16)        0         
                                                                 
 global_average_pooling2d_7   (None, 16)               0         
 (GlobalAveragePooling2D)                                        
                                 

INFO:tensorflow:Assets written to: C:\Users\Jimmy\AppData\Local\Temp\tmp35fmgdzh\assets


INFO:tensorflow:Assets written to: C:\Users\Jimmy\AppData\Local\Temp\tmp35fmgdzh\assets



RAM: 50176,	 Flash: 5048,	 MACC: 1080281




{'RAM': 'Outside the upper bound', 'Flash': 5048, 'MACC': 1080281, 'max_val_acc': -3, 'k': 16, 'c': 0}




Resulting architecture: {'RAM': 30720, 'Flash': 7192, 'MACC': 1260281, 'max_val_acc': 0.66, 'k': 8, 'c': 1}

Elapsed time (search): 0:03:41.057630



c:\Users\Jimmy\miniconda3\envs\tf\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


## Second NAS (PSO)

In [66]:
# --- 1. SEARCH SPACE & ENCODING ---
# We represent an architecture as a vector: [k, c]
# k: Initial number of filters (Continuous, then rounded to int)
# c: Number of additional cells (Continuous, then rounded to int)

class ArchitectureSearchSpace:
    def __init__(self, k_range=(4, 128), c_range=(0, 10)):
        self.k_min, self.k_max = k_range
        self.c_min, self.c_max = c_range

    def clamp(self, k, c):
        """Ensures particles stay within the defined search space."""
        return np.clip(k, self.k_min, self.k_max), np.clip(c, self.c_min, self.c_max)

# --- 2. DECODER (Model Creator) ---
class ModelDecoder:
    def __init__(self):
        self.input_shape = None
        self.num_classes = None
        self.learning_rate = None

    def decode_and_build(self, k, c):
        """Transforms PSO coordinates into a Keras model + MAC count."""
        k, c = int(k), int(c)
        kernel_size = (3, 3)
        pool_size = (2, 2)
        
        number_of_mac = 0
        number_of_cells_limited = False
        
        inputs = keras.Input(shape=self.input_shape)
        n = k
        multiplier = 2
        
        # First Layer
        c_in = self.input_shape[2]
        x = keras.layers.Conv2D(n, kernel_size, padding='same')(inputs)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.ReLU()(x)
        
        # Simplified MAC calculation for clarity
        number_of_mac += (c_in * np.prod(kernel_size) * x.shape[1] * x.shape[2] * x.shape[3])

        # Adding Cells
        for i in range(1, c + 1):
            if x.shape[1] <= 1 or x.shape[2] <= 1:
                number_of_cells_limited = True
                break
            
            n = int(np.ceil(n * multiplier))
            multiplier -= 2**-i
            x = keras.layers.MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='valid')(x)
            
            c_in = x.shape[3]
            x = keras.layers.Conv2D(n, kernel_size, padding='same')(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.ReLU()(x)
            number_of_mac += (c_in * np.prod(kernel_size) * x.shape[1] * x.shape[2] * x.shape[3])

        # Classifier
        x = keras.layers.GlobalAveragePooling2D()(x)
        feat_dim = x.shape[1]
        x = keras.layers.Dense(n)(x)
        number_of_mac += (feat_dim * n)
        x = keras.layers.Dense(self.num_classes)(x)
        outputs = keras.layers.Softmax()(x)
        number_of_mac += (n * self.num_classes)

        model = keras.Model(inputs=inputs, outputs=outputs)
        opt = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        
        return model, number_of_mac, number_of_cells_limited

# --- 3. EVALUATOR & CONSTRAINTS ---
class NASPsoOptimizer:
    def __init__(self, evaluate_model_fnc, input_shape, num_classes, learning_rate):
        self.evaluate_model_fnc = evaluate_model_fnc # External fnc that returns {'max_val_acc': float}
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.model_name = ""
        
        self.setup_decoder()
        
    def setup_decoder(self):
        print(type(self.decoder))
        self.decoder.input_shape=self.input_shape
        self.decoder.num_classes=self.num_classes
        self.decoder.learning_rate=self.learning_rate

    def search(self):
        # Initialize particles [k, c] and velocities
        particles = np.array([
            [np.random.uniform(self.space.k_min, self.space.k_max), 
             np.random.uniform(self.space.c_min, self.space.c_max)] 
            for _ in range(self.n_particles)
        ])
        velocities = np.zeros((self.n_particles, 2))
        
        p_best = np.copy(particles)
        p_best_scores = np.full(self.n_particles, -1.0)
        
        g_best = None
        g_best_score = -1.0

        w, c1, c2 = 0.5, 1.5, 1.5 # Hyperparameters for PSO

        start = datetime.datetime.now()
        
        results_best = {}
        for it in range(self.iterations):
            print(f"==================== iteration {it} ====================")
            for i in range(self.n_particles):
                k, c = particles[i]
                
                # Build and Evaluate
                model, macc, limited = self.decoder.decode_and_build(k, c)
                
                # Constraint Check
                if limited:
                    score = 0 # Penalty for invalid architectures
                else:
                    self.model_name = f"k_{int(k)}_c_{int(c)}"
                    results = self.evaluate_model_fnc(model, macc, limited, self.model_name)
                    score = results['max_val_acc']

                # Update Personal Best
                if score > p_best_scores[i]:
                    p_best_scores[i] = score
                    p_best[i] = particles[i]

                # Update Global Best
                if score > g_best_score:
                    results_best = results
                    g_best_score = score
                    g_best = np.copy(particles[i])

            # Update Velocities and Positions
            if g_best is None:
                # Option A: If no valid architecture was found, re-randomize or skip update
                print("Warning: No valid architecture found in this iteration. Re-randomizing velocities...")
                velocities = np.random.uniform(-1, 1, size=velocities.shape)
            else:
                # Standard PSO Update logic
                for i in range(self.n_particles):
                    r1, r2 = np.random.rand(), np.random.rand()
                    velocities[i] = (w * velocities[i] + 
                                    c1 * r1 * (p_best[i] - particles[i]) + 
                                    c2 * r2 * (g_best - particles[i])) # No longer crashes
                    
                    particles[i] += velocities[i]
                    particles[i][0], particles[i][1] = self.space.clamp(particles[i][0], particles[i][1])

                print(f"Iteration {it}: Global Best Score = {g_best_score:.4f} at k={int(g_best[0])}, c={int(g_best[1])}")

        results_best["k"] = int(g_best[0])
        results_best["c"] = int(g_best[1])
        
        end = datetime.datetime.now()
        return results_best, end-start
    
    def setup(search_space, decoder,  n_particles=5, iterations=10):
        NASPsoOptimizer.n_particles = n_particles
        NASPsoOptimizer.iterations = iterations
        NASPsoOptimizer.space = search_space
        NASPsoOptimizer.decoder = decoder
        return NASPsoOptimizer

In [22]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos.tar', origin=dataset_url, extract=True)
data_dir = Path(data_dir).with_suffix('')

In [68]:
input_shape = (50,50,3)

#target: STM32L412KBU3
#273 CoreMark, 40 kiB RAM, 128 kiB Flash
peak_RAM_upper_bound = 40960
Flash_upper_bound = 131072
MACC_upper_bound = 2730000 #CoreMark * 1e4

#Each dataset must comply with the following structure
#main_directory/
#...class_a/
#......a_image_1.jpg
#......a_image_2.jpg
#...class_b/
#......b_image_1.jpg
#......b_image_2.jpg
path_to_training_set = data_dir
val_split = 0.3

#whether or not to cache datasets in memory
#if the dataset cannot fit in the main memory, the application will crash
cache = True

#where to save results
save_path = ''

#to show the GPU used
!nvidia-smi




search_space = ArchitectureSearchSpace(k_range=(2, 10), c_range=(1, 5))
decoder = ModelDecoder()

# Pass your existing evaluation logic here
colabNAS = ColabNAS(peak_RAM_upper_bound, Flash_upper_bound, MACC_upper_bound, path_to_training_set, val_split, cache, input_shape, save_path=save_path)
#search
path_to_tflite_model = colabNAS.search(NASPsoOptimizer.setup(search_space, decoder))

Tue Jan  6 03:50:54 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 591.59                 Driver Version: 591.59         CUDA Version: 13.1     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
| 47%   40C    P8             29W /  350W |   11635MiB /  12288MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

<class '__main__.ModelDecoder'>
==================== iteration 0 ====================

k_8_c_3

21/21 [==============================] - 2s 35ms/step - loss: 1.3344 - accuracy: 0.4585 - val_loss: 3.1211 - val_accuracy: 0.2598


INFO:tensorflow:Assets written to: C:\Users\Jimmy\AppData\Local\Temp\tmp5hiebvn7\assets


INFO:tensorflow:Assets written to: C:\Users\Jimmy\AppData\Local\Temp\tmp5hiebvn7\assets
c:\Users\Jimmy\miniconda3\envs\tf\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "



RAM: 31744,	 Flash: 20896,	 MACC: 1991994

Epoch 1/99
19/21 [==========================>...] - ETA: 0s - loss: 1.0918 - accuracy: 0.5724
Epoch 1: val_accuracy improved from -inf to 0.33878, saving model to trained_models\k_8_c_3.h5
21/21 [==============================] - 0s 19ms/step - loss: 1.0902 - accuracy: 0.5707 - val_loss: 1.9871 - val_accuracy: 0.3388
Epoch 2/99
18/21 [========================>.....] - ETA: 0s - loss: 1.0229 - accuracy: 0.6007
Epoch 2: val_accuracy improved from 0.33878 to 0.35332, saving model to trained_models\k_8_c_3.h5
21/21 [==============================] - 0s 14ms/step - loss: 1.0206 - accuracy: 0.6030 - val_loss: 1.8477 - val_accuracy: 0.3533
Epoch 3/99
19/21 [==========================>...] - ETA: 0s - loss: 0.9734 - accuracy: 0.6242
Epoch 3: val_accuracy improved from 0.35332 to 0.39873, saving model to trained_models\k_8_c_3.h5
21/21 [==============================] - 0s 13ms/step - loss: 0.9717 - accuracy: 0.6240 - val_loss: 1.5381 - val_accuracy: 

...

INFO:tensorflow:Assets written to: C:\Users\Jimmy\AppData\Local\Temp\tmpm06b1g59\assets
c:\Users\Jimmy\miniconda3\envs\tf\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "



RAM: 33792,	 Flash: 13840,	 MACC: 2149470

Epoch 1/99
14/21 [===================>..........] - ETA: 0s - loss: 1.2649 - accuracy: 0.5073
Epoch 1: val_accuracy improved from -inf to 0.28974, saving model to trained_models\k_9_c_2.h5
21/21 [==============================] - 0s 17ms/step - loss: 1.2518 - accuracy: 0.5111 - val_loss: 2.2134 - val_accuracy: 0.2897
Epoch 2/99
18/21 [========================>.....] - ETA: 0s - loss: 1.1551 - accuracy: 0.5386
Epoch 2: val_accuracy improved from 0.28974 to 0.36785, saving model to trained_models\k_9_c_2.h5
21/21 [==============================] - 0s 10ms/step - loss: 1.1561 - accuracy: 0.5391 - val_loss: 1.7744 - val_accuracy: 0.3678
Epoch 3/99
19/21 [==========================>...] - ETA: 0s - loss: 1.1005 - accuracy: 0.5650
Epoch 3: val_accuracy improved from 0.36785 to 0.41689, saving model to trained_models\k_9_c_2.h5
21/21 [==============================] - 0s 10ms/step - loss: 1.1005 - accuracy: 0.5621 - val_loss: 1.4840 - val_accuracy: 

INFO:tensorflow:Assets written to: C:\Users\Jimmy\AppData\Local\Temp\tmps4pk56_0\assets


INFO:tensorflow:Assets written to: C:\Users\Jimmy\AppData\Local\Temp\tmps4pk56_0\assets
c:\Users\Jimmy\miniconda3\envs\tf\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


Iteration 9: Global Best Score = 0.7080 at k=10, c=2

Resulting architecture: {'RAM': 36352, 'Flash': 15584, 'MACC': 2578650, 'max_val_acc': 0.708, 'k': 10, 'c': 2}

Elapsed time (search): 0:18:59.180376



### Test the obtained model

a useful function for testing 'tflite' models

test the obtained model

In [69]:
_, test_ds = colabNAS.get_data()

In [70]:
def test_tflite_model(path_to_resulting_architecture, test_ds):
    # Convert the Path object to a string
    interpreter = tf.lite.Interpreter(model_path=str(path_to_resulting_architecture))
    interpreter.allocate_tensors()

    output = interpreter.get_output_details()[0]  # Model has single output.
    input = interpreter.get_input_details()[0]  # Model has single input.
    input_dtype = input['dtype']

    correct = 0
    wrong = 0

    for images_batch, labels_batch in test_ds:
        for image, label in zip(images_batch, labels_batch):
            # Check if the input type is quantized, then rescale input data to uint8
            if input_dtype == np.uint8 or input_dtype == tf.uint8:
                input_scale, input_zero_point = input["quantization"]
                image = image / input_scale + input_zero_point
            input_data = np.expand_dims(image.numpy().astype(input_dtype), axis=0)
            interpreter.set_tensor(input['index'], input_data)
        interpreter.invoke()
        if label.numpy().argmax() == interpreter.get_tensor(output['index']).argmax() :
            correct = correct + 1
        else :
            wrong = wrong + 1
    print(f"\nTflite model test accuracy: {correct/(correct+wrong)}")

In [72]:
test_tflite_model(path_to_tflite_model, test_ds)


Tflite model test accuracy: 0.7777777777777778
